In [119]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
import time
import re
import datetime

# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежими новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [9]:
KEYWORDS = ['python', 'парсинг']
URL_HABR_NEWS = 'https://habr.com/ru/news/'
MAX_PAGE = 50  # Последняя страница всегда 50-ая

In [65]:
def check_keywords_in_text(text, keywords):
    text_words = re.split('\s', text.lower())
    for word in keywords:
        if word in text_words:
            return True
    return False

In [148]:
def get_habr_news():
    list_news = []
    for i in range(1, MAX_PAGE+1):
        print(f'Страница: {i}')
        time.sleep(1)
        date = datetime.datetime.now().date()
        res = requests.get(f'{URL_HABR_NEWS}page{i}')
        soup = BeautifulSoup(res.text, 'html.parser')
        soap_articles = soup.find_all('article', class_='post post_preview')
        
        for articles in soap_articles:
            news = {}
            news['request_dt'] = date
            news['article_dt'] = articles.find('span', class_='post__time').text
            news['article_header'] = articles.find('h2').text
            news['article_preview'] = articles.find('div', class_='post__text').text
            news['article_ref'] = articles.find('h2').find('a').get('href')
        
            list_news.append(news)
    
    return list_news

In [164]:
def filter_habr_news_by_keywords(news, keywords):
    filter_news = []
    for article in news:
        text_header = article['article_header']
        text_preview = article['article_preview']
        text_content = article.get('content', '')

        if check_keywords_in_text(text_header, keywords) or \
            check_keywords_in_text(text_preview, keywords) or \
            check_keywords_in_text(text_content, keywords):
            filter_news.append(article)
    return filter_news

In [213]:
def transformation_date_article(article_dt, request_dt):
    months = {'января':'Jan',
         'февраля':'Feb',
         'марта':'Mar',
         'апреля':'Apr',
         'мая':'May',
         'июня':'Jun',
         'июля':'Jul',
         'августа':'Aug',
         'сентября':'Sep',
         'октября':'Oct',
         'ноября':'Nov',
         'декабря':'Dec'}
    
    if 'сегодня' in article_dt:
        dt = request_dt.strftime('%Y-%m-%d')
    
    elif 'вчера' in article_dt:
        dt = (request_dt - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    
    else:
        dt_list = article_dt.split(' ')[:3]
        dt = datetime.datetime.strptime(dt_list[0] + ' ' 
                                        + months[f'{dt_list[1]}'] + ' ' 
                                        + dt_list[2], '%d %b %Y').strftime('%Y-%m-%d')
    
    return dt

In [214]:
news = get_habr_news()

Страница: 1
Страница: 2
Страница: 3
Страница: 4
Страница: 5
Страница: 6
Страница: 7
Страница: 8
Страница: 9
Страница: 10
Страница: 11
Страница: 12
Страница: 13
Страница: 14
Страница: 15
Страница: 16
Страница: 17
Страница: 18
Страница: 19
Страница: 20
Страница: 21
Страница: 22
Страница: 23
Страница: 24
Страница: 25
Страница: 26
Страница: 27
Страница: 28
Страница: 29
Страница: 30
Страница: 31
Страница: 32
Страница: 33
Страница: 34
Страница: 35
Страница: 36
Страница: 37
Страница: 38
Страница: 39
Страница: 40
Страница: 41
Страница: 42
Страница: 43
Страница: 44
Страница: 45
Страница: 46
Страница: 47
Страница: 48
Страница: 49
Страница: 50


In [306]:
filter_news = filter_habr_news_by_keywords(news, KEYWORDS)
for article in filter_news:
    article['article_dt_trans'] = transformation_date_article(article['article_dt'], article['request_dt'])

In [308]:
df = pd.DataFrame(filter_news, columns=['article_dt_trans', 'request_dt', 'article_dt', 'article_header', 'article_preview', 'article_ref'])
df.drop(['request_dt', 'article_preview', 'article_dt'], axis=1)

,article_dt_trans,article_header,article_ref
0,2021-02-01,\nMail.ru Group открывает набор на бесплатные ...,https://habr.com/ru/company/mailru/news/t/540268/
1,2020-12-14,\nГлубокое обучение на Kotlin: вышла альфа-вер...,https://habr.com/ru/company/JetBrains/news/t/5...


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [170]:
def get_habr_news_full_text(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    return soup.find('div', class_='post__text').text

In [220]:
news_full = [] 
for article in news:
    print(article['article_ref'])
    article_full = {}
    while True:
        try:
            time.sleep(1)
            article_full['request_dt'] = article['request_dt']
            article_full['article_dt'] = article['article_dt']
            article_full['article_header'] = article['article_header']
            article_full['article_preview'] = article['article_preview']
            article_full['article_ref'] = article['article_ref']
            article_full['content'] = get_habr_news_full_text(article['article_ref'])
            break
        except Exception as er:
            print(er)
    news_full.append(article_full)

https://habr.com/ru/news/t/542342/
https://habr.com/ru/news/t/542332/
https://habr.com/ru/news/t/542320/
https://habr.com/ru/news/t/542318/
https://habr.com/ru/news/t/542310/
https://habr.com/ru/news/t/542306/
https://habr.com/ru/news/t/542268/
https://habr.com/ru/news/t/542240/
https://habr.com/ru/news/t/542230/
https://habr.com/ru/news/t/542236/
https://habr.com/ru/news/t/542232/
https://habr.com/ru/news/t/542228/
https://habr.com/ru/news/t/542222/
https://habr.com/ru/news/t/542186/
https://habr.com/ru/news/t/542164/
https://habr.com/ru/news/t/542146/
https://habr.com/ru/news/t/542124/
https://habr.com/ru/news/t/542112/
https://habr.com/ru/news/t/542100/
https://habr.com/ru/news/t/542092/
https://habr.com/ru/news/t/542072/
https://habr.com/ru/news/t/542060/
https://habr.com/ru/news/t/542054/
https://habr.com/ru/news/t/542056/
https://habr.com/ru/news/t/542050/
https://habr.com/ru/news/t/542046/
https://habr.com/ru/news/t/542016/
https://habr.com/ru/news/t/542006/
https://habr.com/ru/

https://habr.com/ru/news/t/540006/
https://habr.com/ru/news/t/540004/
https://habr.com/ru/news/t/539998/
https://habr.com/ru/news/t/539996/
https://habr.com/ru/news/t/539994/
https://habr.com/ru/news/t/539990/
https://habr.com/ru/company/sberbank/news/t/539980/
https://habr.com/ru/news/t/539972/
https://habr.com/ru/news/t/539952/
https://habr.com/ru/news/t/539950/
https://habr.com/ru/news/t/539942/
https://habr.com/ru/news/t/539934/
https://habr.com/ru/news/t/539926/
https://habr.com/ru/news/t/539922/
https://habr.com/ru/news/t/539916/
https://habr.com/ru/news/t/539912/
https://habr.com/ru/news/t/539902/
https://habr.com/ru/news/t/539896/
https://habr.com/ru/news/t/539886/
https://habr.com/ru/news/t/539872/
https://habr.com/ru/news/t/539880/
https://habr.com/ru/news/t/539876/
https://habr.com/ru/news/t/539870/
https://habr.com/ru/news/t/539854/
https://habr.com/ru/news/t/539842/
https://habr.com/ru/news/t/539838/
https://habr.com/ru/news/t/539806/
https://habr.com/ru/news/t/539802/
htt

https://habr.com/ru/news/t/537996/
https://habr.com/ru/company/gitlab/news/t/537952/
https://habr.com/ru/news/t/537940/
https://habr.com/ru/company/proto/news/t/537918/
https://habr.com/ru/news/t/537912/
https://habr.com/ru/news/t/537878/
https://habr.com/ru/news/t/537870/
https://habr.com/ru/news/t/537852/
https://habr.com/ru/news/t/537844/
https://habr.com/ru/company/intersect/news/t/537842/
https://habr.com/ru/news/t/537828/
https://habr.com/ru/news/t/537832/
https://habr.com/ru/news/t/537830/
https://habr.com/ru/news/t/537824/
https://habr.com/ru/news/t/537818/
https://habr.com/ru/news/t/537816/
https://habr.com/ru/news/t/537810/
https://habr.com/ru/news/t/537804/
https://habr.com/ru/news/t/537806/
https://habr.com/ru/company/southbridge/news/t/537798/
https://habr.com/ru/news/t/537802/
https://habr.com/ru/news/t/537788/
https://habr.com/ru/news/t/537746/
https://habr.com/ru/news/t/537744/
https://habr.com/ru/news/t/537702/
https://habr.com/ru/news/t/537710/
https://habr.com/ru/new

https://habr.com/ru/news/t/535598/
https://habr.com/ru/news/t/535566/
https://habr.com/ru/company/ruvds/news/t/535504/
https://habr.com/ru/news/t/535500/
https://habr.com/ru/news/t/535494/
https://habr.com/ru/news/t/535482/
https://habr.com/ru/company/google/news/t/535478/
https://habr.com/ru/news/t/535434/
https://habr.com/ru/company/it-grad/news/t/535414/
https://habr.com/ru/news/t/535408/
https://habr.com/ru/news/t/535350/
https://habr.com/ru/news/t/535346/
https://habr.com/ru/news/t/535320/
https://habr.com/ru/company/alconost/news/t/535318/
https://habr.com/ru/news/t/535314/
https://habr.com/ru/news/t/535312/
https://habr.com/ru/company/cloud4y/news/t/535300/
https://habr.com/ru/news/t/535272/
https://habr.com/ru/news/t/535276/
https://habr.com/ru/news/t/535194/
https://habr.com/ru/news/t/535190/
https://habr.com/ru/news/t/535172/
https://habr.com/ru/news/t/535146/
https://habr.com/ru/news/t/535122/
https://habr.com/ru/news/t/535116/
https://habr.com/ru/news/t/535108/
https://habr

HTTPSConnectionPool(host='habr.com', port=443): Max retries exceeded with url: /ru/news/t/533190/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001993F16C808>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))
HTTPSConnectionPool(host='habr.com', port=443): Max retries exceeded with url: /ru/news/t/533190/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001993F192B48>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))
HTTPSConnectionPool(host='habr.com', port=443): Ma

In [309]:
filter_news = filter_habr_news_by_keywords(news_full, KEYWORDS)
for article in filter_news:
    article['article_dt_trans'] = transformation_date_article(article['article_dt'], article['request_dt'])

In [310]:
df = pd.DataFrame(filter_news, columns=['article_dt_trans', 'request_dt', 'article_dt', 'article_header', 'article_preview', 'article_ref', 'content'])
df.drop(['request_dt', 'article_preview', 'article_dt'], axis=1)

,article_dt_trans,article_header,article_ref,content
0,2021-02-10,\nИсследователь взломал системы 35 крупных IT-...,https://habr.com/ru/news/t/541784/,Исследователю Алексу Бирсану удалось взломать ...
1,2021-02-01,\nMail.ru Group открывает набор на бесплатные ...,https://habr.com/ru/company/mailru/news/t/540268/,Мы запускаем два бесплатных онлайн-курса: по с...
2,2021-02-01,\nMozilla закрывает проекты Firefox Voice и Vo...,https://habr.com/ru/company/itsumma/news/t/540...,Компания Mozilla объявила о закрытии своих сер...
3,2021-01-25,\nОткрываем четыре Школы: разработки интерфейс...,https://habr.com/ru/company/yandex/news/t/538798/,"В 2012 году трое руководителей разработки, вкл..."
4,2020-12-17,"\nВ МТИ показали, как мозг читает компьютерный...",https://habr.com/ru/news/t/533518/,\n\r\nНейробиологи из Массачусетского технолог...
5,2020-12-14,\nГлубокое обучение на Kotlin: вышла альфа-вер...,https://habr.com/ru/company/JetBrains/news/t/5...,Всем привет! \nНа днях мы выпустили первую аль...


## Задание 2

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<почта> - <дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [230]:
EMAIL = ['matlserg@mail.ru', 'matlserg@gmail.com', 'matlserg@yandex.ru']

In [303]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

headers = {
    'Vaar-Version':'0',
    'Vaar-Header-App-Product':'hackcheck-web-avast'
}

payload = {'emailAddresses':EMAIL}

res = requests.post(url, json=payload, headers=headers)
data = res.json()

In [304]:
desc_breaches = []
for email, breaches in data['summary'].items():
    for breach_id in breaches['breaches']:
        breach = {}
        breach['email'] = email
        breach['Id'] = breach_id
        
        desc_breaches.append(breach)

for breach in desc_breaches:
    desc = data['breaches'][f'{breach["Id"]}']
    breach['source'] = desc['site']
    breach['date'] = datetime.datetime.strptime(desc['publishDate'], "%Y-%m-%dT%H:%M:%SZ").\
                                                                                strftime('%Y-%m-%d')
    breach['desc'] = desc['description']

In [305]:
df = pd.DataFrame(desc_breaches, columns=['email', 'date', 'source', 'desc'])
df

,email,date,source,desc
0,matlserg@mail.ru,2019-03-28,verifications.io,Big data e-mail verification platform verifica...
1,matlserg@mail.ru,2016-11-01,qip.ru,"In 2011, Russian instant messaging service pro..."
2,matlserg@mail.ru,2017-11-04,myheritage.com,"In October 2017, a customer database belonging..."
3,matlserg@mail.ru,2016-10-21,adobe.com,"In October of 2013, criminals penetrated Adobe..."
4,matlserg@mail.ru,2021-02-04,rzd-bonus.ru,"In June 2020, the Russian Railways Bonus Progr..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
